In [37]:
# Local application imports
from utils import postgresListDatabases, postgresListTables, postgresImportFile, postgresRunSqlScript, postgresGetCredentials

import pandas as pd
from sqlalchemy import create_engine
from sklearn.neighbors import KDTree

In [38]:
db_name='weather_wizards'

sql_file = 'sql/updated_schema.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

sql_file = 'sql/swdiws_nx3tvs.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

table_name = 'swdiws_nx3tvs_geojson'
file_location = 'data/swdiws_nx3tvs_geojson.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")     

table_name = 'radar_station'
file_location = 'data/radar_locations.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")

databases = postgresListDatabases()
for database in databases:
    print(f"{database}")
    tables = postgresListTables(database)
    for table in tables:
        print(f"\t{table}")

Successfully executed 16 commands from sql/updated_schema.sql.
Successfully executed 2 commands from sql/swdiws_nx3tvs.sql.
Loaded ('Successfully imported 2500 records from data/swdiws_nx3tvs_geojson.csv into swdiws_nx3tvs_geojson.', 2500) records to swdiws_nx3tvs_geojson
Loaded ('Successfully imported 192 records from data/radar_locations.csv into radar_station.', 192) records to radar_station
city_info
	cities
	movie_words_comparison
	people
	programming_languages
programming_db
movie
	movie_table
crud
act2
	actor
	evr-import
pets_db
	owners
	pet_names
	pet_names_and_owners
	pet_names_updated
	service
foreignk_db
	customer
	customer_email
	customer_phone
relationship_db
	courses
	student_courses_junction
	students
weather_wizards
	hurricane_archive_summary_db
	hurricane_archive_summary_staging
	hurricane_details_staging
	hurricane_summary
	hurricane_summary_staging
	impact_type
	radar_station
	region
	swdiws_nx3tvs_geojson
	weather_event
	weather_event_cost_impact
	weather_event_huma

In [39]:
db_name='weather_wizards'

sql_file = 'sql/hurricane_summary_staging.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

Successfully executed 2 commands from sql/hurricane_summary_staging.sql.


In [40]:
table_name = 'hurricane_summary_staging'
file_location = 'data/hurricane_summary.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")

Loaded ('Successfully imported 901 records from data/hurricane_summary.csv into hurricane_summary_staging.', 901) records to hurricane_summary_staging


In [41]:
db_name='weather_wizards'

sql_file = 'sql/hurricane_details_staging.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

Successfully executed 2 commands from sql/hurricane_details_staging.sql.


In [42]:
table_name = 'hurricane_details_staging'
file_location = 'data/hurricane_details.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")

Loaded ('Successfully imported 5001 records from data/hurricane_details.csv into hurricane_details_staging.', 5001) records to hurricane_details_staging


In [43]:
file_location = 'data/hurricane_details_retired.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")

Loaded ('Successfully imported 7917 records from data/hurricane_details_retired.csv into hurricane_details_staging.', 7917) records to hurricane_details_staging


In [44]:
sql_file = 'sql/hurricane_archive_summary_staging.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

Successfully executed 2 commands from sql/hurricane_archive_summary_staging.sql.


In [45]:
table_name = 'hurricane_archive_summary_staging'
file_location = 'data/hurricane_archive_summary.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")

Loaded ('Successfully imported 168 records from data/hurricane_archive_summary.csv into hurricane_archive_summary_staging.', 168) records to hurricane_archive_summary_staging


In [46]:
# https://simplemaps.com/data/us-zips

db_name='weather_wizards'

engine = create_engine(postgresGetCredentials(engine_type='sqlalchemy') + '/' + db_name)

# Load the dataset
zip_data = pd.read_csv('data/uszips.csv')
# zip_data.info()
zip_data_region = (
    zip_data[['city', 'state_id', 'zip', 'population', 'lat', 'lng']]
    .assign(
        # Fill NaN in 'population' with 0 and convert to int
        population=lambda zip_data: zip_data['population'].fillna(0).astype(int),
        # Fill NaN in 'zip' with 0, convert to int, then to string and zero-fill to 5 digits
        zip=lambda zip_data: zip_data['zip'].fillna(0).astype(int).astype(str).str.zfill(5)
    )
    .rename(columns={'state_id': 'state', 'zip_code': 'zip', 'population': 'population_size', 'lat': 'latitude', 'lng': 'longitude'})
)

zip_data_region.info()

zip_data_region.to_sql('region', con=engine, if_exists='append', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33784 entries, 0 to 33783
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city             33784 non-null  object 
 1   state            33784 non-null  object 
 2   zip              33784 non-null  object 
 3   population_size  33784 non-null  int32  
 4   latitude         33784 non-null  float64
 5   longitude        33784 non-null  float64
dtypes: float64(2), int32(1), object(3)
memory usage: 1.4+ MB


784